In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# https://drive.google.com/file/d/1ztp2g83wBuCJOCRmj1jGxlsKdPwfFW6A/view?usp=sharing
read_file = drive.CreateFile({'id': '1ztp2g83wBuCJOCRmj1jGxlsKdPwfFW6A'})

read_file.GetContentFile('Test_Data_ second_test_case.xlsx')

import pandas as pd
import re
import datetime
from datetime import datetime
import inflect
i_eng = inflect.engine()

source_data = pd.read_excel('Test_Data_ second_test_case.xlsx')

In [ ]:
# source_data

In [ ]:
startTime = datetime.now()

#Create a copy of dataframe 
source_data_copy =  source_data.copy()

# Dropping rows with null value in the 'value' column containing current period data points   
source_data_copy.dropna(subset=['value'], inplace= True)

# handling 'nan' value in the comparison column
source_data_copy['ComparisonPeriod'] = source_data_copy['ComparisonPeriod'].fillna(0)

# handling 'nan' value in the multiplierCode column
source_data_copy['multiplierCode'] = source_data_copy['multiplierCode'].fillna('in the amount of')

# handling 'nan' value in the currencycode column
source_data_copy['currencycode'] = source_data_copy['currencycode'].fillna(' ')

# checking for rows with value 0 in the 'value' column containing current period data points
# if the current data value is 0 data should not be processed
# source_data_copy.loc[source_data_copy['value'] == 0.0]
# 
# creating list of indices with value 0.0
# (source_data_copy['value'] == 0.0).value_counts()

# removing rows with value 0 in the 'value' column containing current period data points   
# test code: source_data_r = source_data_copy[source_data_copy.value != 0.0] 
source_data_copy = source_data_copy[source_data_copy['value'] != 0.0]

# Creating column to conver period in text form for headline and situation
source_data_copy['text_period'] = None 
# Text for periods
x = source_data_copy.loc[lambda source_data_copy: source_data_copy['period']== 3.0].index
for i in x:
    source_data_copy.loc[i, 'text_period'] = 'quarter'
y = source_data_copy.loc[lambda source_data_copy: source_data_copy['period']== 6.0].index
for i in y:
    source_data_copy.loc[i, 'text_period'] = 'half year'
z = source_data_copy.loc[lambda source_data_copy: source_data_copy['period']== 9.0].index
for i in z:
    source_data_copy.loc[i, 'text_period'] = 'nine months'
f = source_data_copy.loc[lambda source_data_copy: source_data_copy['period']== 12.0].index
for i in f:
    source_data_copy.loc[i, 'text_period'] = 'full year'
# for odd periods changing none fields in 'text_period' to 'period'

# Converting PEO dates 
source_data_copy['con_PEO'] = None 
source_data_copy['con_PEO'] = pd.to_datetime(source_data_copy.PEO)
source_data_copy['con_PEO'] = source_data_copy['con_PEO'].dt.strftime('%B %d, %Y')

# Converting fiscal quarter to text
source_data_copy['text_fiscalquarter'] = None 
x = source_data_copy.loc[lambda source_data_copy: source_data_copy['FiscalQuarter']== 1.0].index
for i in x:
    source_data_copy.loc[i, 'text_fiscalquarter'] = 'first'
y = source_data_copy.loc[lambda source_data_copy: source_data_copy['FiscalQuarter']== 2.0].index
for i in y:
    source_data_copy.loc[i, 'text_fiscalquarter'] = 'second'
z = source_data_copy.loc[lambda source_data_copy: source_data_copy['FiscalQuarter']== 3.0].index
for i in z:
    source_data_copy.loc[i, 'text_fiscalquarter'] = 'third'
f = source_data_copy.loc[lambda source_data_copy: source_data_copy['FiscalQuarter']== 4.0].index
for i in f:
    source_data_copy.loc[i, 'text_fiscalquarter'] = 'fourth'

#####################################################################################
# converting 'values' to millions and thusands and format

source_data_copy['con_value_fin'] = None
source_data_copy['con_ComparisonPeriod_fin'] = None

list_code = list(source_data_copy['multiplierCode'])
list_uni_code = list(set(list_code))
for code in list_uni_code:
    text_code_df = source_data_copy.loc[lambda source_data_copy: source_data_copy['multiplierCode'] == code]
    code_ind = text_code_df.index
    for ci in code_ind:
        x_val = 0
        if code == 'MLN':
            x_val = text_code_df.loc[ci, 'value']
            source_data_copy.loc[ci, 'con_value_fin'] = '{:,.0f}'.format(x_val) + ' million'
        elif code == 'THO':
            x_val = text_code_df.loc[ci, 'value']*1000
            source_data_copy.loc[ci, 'con_value_fin'] = '{:,.0f}'.format(x_val)
        elif code == 'BLN':
            x_val = text_code_df.loc[ci, 'value']
            source_data_copy.loc[ci, 'con_value_fin'] = '{:,.0f}'.format(x_val) + ' billion'
        elif code == 'UNT':
            x_val = source_data_copy.loc[ci, 'value']
            source_data_copy.loc[ci, 'con_value_fin'] = '{:,.0f}'.format(x_val)
        elif code == 'in the amount of':
# Handling null multiplierCode chnaged to 'in the amount of'
#>> Keeping "source_data_copy['multiplierCode'][ci] = 'in the amount of'" to allow comparison value collection- changes to empty apter handling comaprison value
#            source_data_copy['multiplierCode'][ci] = ' '
            x_val = source_data_copy.loc[ci, 'value']
            source_data_copy.loc[ci, 'con_value_fin'] = '{:,.0f}'.format(x_val)
            
    for ci in code_ind:
        if 'per share' in source_data_copy.loc[ci, 'CurrentQAccountName']:
            cval = float(source_data_copy.loc[ci, 'value'])
            source_data_copy.loc[ci, 'con_value_fin'] = cval
        else:
            nrq = 1

for code in list_uni_code:
    text_code_df = source_data_copy.loc[lambda source_data_copy: source_data_copy['multiplierCode'] == code]
    code_ind = text_code_df.index
    for ci in code_ind:
        x = 0
        if code == 'MLN':
            x = text_code_df.loc[ci, 'ComparisonPeriod']
            source_data_copy.loc[ci, 'con_ComparisonPeriod_fin'] = '{:,.0f}'.format(x) + ' million'
        elif code == 'THO':
            x = text_code_df.loc[ci, 'ComparisonPeriod']*1000
            source_data_copy.loc[ci, 'con_ComparisonPeriod_fin'] = '{:,.0f}'.format(x)
        elif code == 'BLN':
            x = text_code_df.loc[ci, 'ComparisonPeriod']
            source_data_copy.loc[ci, 'con_ComparisonPeriod_fin'] = '{:,.0f}'.format(x) + ' billion'
        elif code == 'UNT':
            x = source_data_copy.loc[ci, 'ComparisonPeriod']
            source_data_copy.loc[ci, 'con_ComparisonPeriod_fin'] = '{:,.0f}'.format(x)
        elif code == 'in the amount of':
# Handling null multiplierCode chnaged to 'in the amount of'
# >> changing 'multiplierCode' here
            source_data_copy.loc[ci, 'multiplierCode'] = ' '
            x = source_data_copy.loc[ci, 'ComparisonPeriod']
            source_data_copy.loc[ci, 'con_ComparisonPeriod_fin'] = '{:,.0f}'.format(x)

    for ci in code_ind:
        if 'per share' in source_data_copy.loc[ci, 'ComparisonQAccountName']:
            comval = float(source_data_copy.loc[ci, 'ComparisonPeriod'])
            source_data_copy.loc[ci, 'con_ComparisonPeriod_fin'] = comval
        else:
            nrq = 1
###### Feature for 'CurrentQAccountName' length
# Creating column
source_data_copy['len_CurrentQAccountName'] = None 
# 
c = list(source_data_copy.index)
for p in c:
    source_data_copy.loc[p, 'len_CurrentQAccountName'] = len(source_data_copy.loc[p, 'CurrentQAccountName'])


##########################################################################################

##   adding columns in dataframe   
source_data_copy['result_headline'] = None
source_data_copy['result_situation']= None

# pulling a list of unique CIDs
source_data_copy['CID']
list_CID =  list(source_data_copy['CID'])
list_uni_CID = list(set(list_CID))

# Data frame based on Company ID
for CoID in list_uni_CID:
    text_CID_df = source_data_copy.loc[lambda source_data_copy: source_data_copy['CID'] == CoID]
#    print(CoID)
#  pulling a list of documents for a CID
    list_doc = list(text_CID_df['CIQDocumentId'])
    list_uni_doc = list(set(list_doc))
    list_uni_doc = [int(x) for x in list_uni_doc]
#  dataframe based on document
    for doc in list_uni_doc:
        text_doc_df = text_CID_df.loc[lambda text_CID_df: text_CID_df['CIQDocumentId'] == doc]
    #  pulling a list of years 
        list_year =  list(text_doc_df['CurrentYear'])
        list_uni_year = list(set(list_year))
        list_uni_year = [int(x) for x in list_uni_year]
    #  dataframe based on year
        for year in list_uni_year:
            text_year_df = text_doc_df.loc[lambda text_doc_df: text_doc_df['CurrentYear'] == year]
    # pulling list of PEO
            list_PEO = list(text_year_df['con_PEO'])
            list_uni_PEO = list(set(list_PEO))
            list_uni_PEO = sorted([x for x in list_uni_PEO])
    #   dataframe based on PEO
            for PEO in list_uni_PEO:
                text_PEO_df = text_year_df.loc[lambda text_year_df: text_year_df['con_PEO']== PEO]
    ##   
    #   Creating lists for headline and situation and appending values stepwise
                gen_text_headline = list()
                gen_text_situation= list()
                comp_ind = text_PEO_df.index[0]
    #       print('CID: ', CoID, '******************************************************************************')
#    df.loc[row, 'column']
                gen_text_headline.append(text_CID_df.loc[comp_ind, 'Companyname'])
                gen_text_headline.append(' ')
                gen_text_headline.append('Announces Earnings Results for the')
                gen_text_headline.append(' ')
                gen_text_situation.append(text_CID_df.loc[comp_ind, 'Companyname'])
                gen_text_situation.append(' ')
                gen_text_situation.append('announced earnings results for the')
                gen_text_situation.append(' ')
    ### pulling list of periods 
                list_period = list(text_PEO_df['period'])
                list_uni_period = list(set(list_period))
                list_uni_period = sorted([int(x) for x in list_uni_period])
    #  # variable for new paragraph's next line
                nxtline_var = 0
    #   dataframe based on period
                for period in list_uni_period:
                    text_period_df = text_PEO_df.loc[lambda text_PEO_df: text_PEO_df['period']== period]
    # adding period in text based on period types, quarterly, yearly, half year, nine month
                    p_ind = text_period_df.index[0]
                    if period == 3:
                        gen_text_headline.append(text_period_df.loc[p_ind, 'text_fiscalquarter'].title())
                        gen_text_headline.append(' ')
                        gen_text_headline.append(text_period_df.loc[p_ind, 'text_period'].title())
                        gen_text_headline.append(' ')
                        gen_text_situation.append(text_period_df.loc[p_ind, 'text_fiscalquarter'])
                        gen_text_situation.append(' ')
                        gen_text_situation.append(text_period_df.loc[p_ind, 'text_period'])
                        gen_text_situation.append(' ')
                        if len(list_uni_period) == 2:
                            gen_text_headline.append('and')
                            gen_text_headline.append(' ')
                            gen_text_situation.append('and')
                            gen_text_situation.append(' ')
                        else: continue
                            
                    elif period == 6:
                        gen_text_headline.append(text_period_df.loc[p_ind, 'text_period'].title())
                        gen_text_headline.append(' ')
                        gen_text_situation.append(text_period_df.loc[p_ind, 'text_period'])
                        gen_text_situation.append(' ')

                    elif period == 9:
                        gen_text_headline.append(text_period_df.loc[p_ind, 'text_period'].title())
                        gen_text_headline.append(' ')
                        gen_text_situation.append(text_period_df.loc[p_ind, 'text_period'])
                        gen_text_situation.append(' ')
                        
                    elif period == 12:
                        gen_text_headline.append(text_period_df.loc[p_ind, 'text_period'].title())
                        gen_text_headline.append(' ')
                        gen_text_situation.append(text_period_df.loc[p_ind, 'text_period'])
                        gen_text_situation.append(' ')
                    else:
                        # for odd periods changing none fields in 'text_period' to 'period'
                        text_PEO_df['text_period'] = text_PEO_df['text_period'].fillna(i_eng.number_to_words(period)+' months')
                        gen_text_headline.append(i_eng.number_to_words(period).title())
                        gen_text_headline.append(' ')
                        gen_text_headline.append('Months ')
                        gen_text_situation.append(i_eng.number_to_words(period))
                        gen_text_situation.append(' ')
                        gen_text_situation.append('months ')


    #   adding period end dates
                gen_text_headline.append('Ended')
                gen_text_headline.append(' ')
                gen_text_headline.append(text_period_df.loc[p_ind, 'con_PEO'])
                gen_text_headline.append(' ')
                gen_text_situation.append('ended')
                gen_text_situation.append(' ')
                gen_text_situation.append(text_period_df.loc[p_ind, 'con_PEO'])
                gen_text_situation.append('.')
                gen_text_situation.append(' ')
    # starting the second line in the siuation    
                for period in list_uni_period:
                    text_period_df = text_PEO_df.loc[lambda text_PEO_df: text_PEO_df['period']== period]
                    p_ind = text_period_df.index[0]
    #            print('Period: ', period, '********* ********************************************************')
    #  Starting additional paragraph in the situation for commulative period
                    nxtline_var = nxtline_var + 1
                    if nxtline_var > 1:
                        gen_text_situation.append('\n')
                    else:
                        mc = 'mch'
                    gen_text_situation.append('For the')
                    gen_text_situation.append(' ')
                    gen_text_situation.append(text_period_df.loc[p_ind, 'text_period'])
                    gen_text_situation.append(', the company reported')
                    gen_text_situation.append(' ')

    # handling same valued basic and diluted EPS and EPS values from continued opertions 
#                    text_period_df.drop_duplicates(subset='con_value_fin', keep="first", inplace= True)
                    text_period_df = text_period_df.sort_values(by='len_CurrentQAccountName', ascending=True)
                    text_period_df = text_period_df.drop_duplicates(subset='con_value_fin', keep="first").sort_index()

    # dropping duplicate sales and revenue values and keeping the maimum value
    #                text_period_df.sort_values('value', ascending=False).drop_duplicates('CurrentQAccountName', inplace=True)
                    text_period_df = text_period_df.sort_values(by='value', ascending=False)
                    text_period_df = text_period_df.drop_duplicates(subset='CurrentQAccountName', keep="first").sort_index()

    #      Collecting datapoints for the situation
                    for row in text_period_df.index:
                        datapoint_name = str(text_period_df.loc[row, 'CurrentQAccountName'])
                        datapoint_currency = str(text_period_df.loc[row, 'currencycode'])
                        datapoint_value = text_period_df.loc[row, 'con_value_fin'] 
                        compare_datapoint_name = str(text_period_df.loc[row, 'ComparisonQAccountName'])
                        compare_datapoint_currency = str(text_period_df.loc[row, 'currencycode'])
                        compare_datapoint_checkvalue= text_period_df.loc[row, 'ComparisonPeriod']
                        compare_datapoint_value = text_period_df.loc[row, 'con_ComparisonPeriod_fin']
                        gen_text_situation.append(datapoint_name.capitalize())
                        gen_text_situation.append(' ')
                        if datapoint_name == 'sales':
                            gen_text_situation.append('were')
                        else:
                            gen_text_situation.append('was')
                        gen_text_situation.append(' ')
    # procssing current period values with '-' sign
                        if '-' in str(datapoint_value):
                            datapoint_value = str(datapoint_value)[1:]
                        else:          
                            datapoint_value = datapoint_value
                        gen_text_situation.append(datapoint_currency)
                        gen_text_situation.append(' ')
                        gen_text_situation.append(datapoint_value)
    # adding text for comparison period
                        if float(compare_datapoint_checkvalue) != 0:
                            gen_text_situation.append(' ')
                            gen_text_situation.append('compared with')
                            gen_text_situation.append(' ')
    # processing cases where data point names are different for current and comparison period like one is 'loss' and other is 'earning' 
                            if datapoint_name == compare_datapoint_name:
                                pass
                            else:
                                gen_text_situation.append(compare_datapoint_name.lower())
                                gen_text_situation.append(' ')
                                gen_text_situation.append('of')
                                gen_text_situation.append(' ')
    # processing comparison values with '-' sign
                            if '-' in str(compare_datapoint_value):
                                compare_datapoint_value = str(compare_datapoint_value)[1:]
                            else:
                                compare_datapoint_value = compare_datapoint_value
                            gen_text_situation.append(compare_datapoint_currency)
                            gen_text_situation.append(' ')
                            gen_text_situation.append(compare_datapoint_value)
                            gen_text_situation.append(' ')
                            gen_text_situation.append('a year ago.')
                            gen_text_situation.append(' ')
                        else:
                            gen_text_situation.append('. ')
    # handling unwanted capitlised data point
                indices = [i for i, y in enumerate(gen_text_situation) if y == ', the company reported']
                for l in indices:
                    posit = l+2
                    gen_text_situation[posit] = gen_text_situation[posit].lower()

            print('**************************************************************************************')
            print('CID: ', CoID, text_CID_df.loc[comp_ind, 'Companyname'])
#           print(gen_text_headline)
#           print(gen_text_situation)
        
# Joining lists, generating strings and adding to dataframe
            print(''.join(map(str, gen_text_headline)))
            print(''.join(map(str, gen_text_situation)))
            source_data_copy.loc[comp_ind, 'result_headline'] = ''.join(map(str, gen_text_headline))
            source_data_copy.loc[comp_ind, 'result_situation']= ''.join(map(str, gen_text_situation))

## Creating dataframe for output
EARfast_result_dataframe = source_data_copy[['CID', 'CIQDocumentId', 'result_headline', 'result_situation', 'filingdate']]
EARfast_result_dataframe.dropna(subset=['result_situation'], inplace= True)
EARfast_result_dataframe.rename(columns={'CID': 'CompanyID', 'CIQDocumentId': 'DocumentID', 'result_headline': 'Headline', 'result_situation': 'Situation', 'filingdate': 'FilingDate'}, inplace=True)

######>>>>>>>>>>>> Exporting to CSV file

EARfast_result_dataframe.to_csv('EARfastrack_result_file2.csv', index=False)
# source_data_copy.to_csv('result_EAR_FileFastrack.csv')
# print('>>>>>>>>>>>>>  Stop  <<<<<<<<<<<<<<<')


print('>>>>>>>', datetime.now() - startTime)

**************************************************************************************
CID:  244277383 Supertec Machinery, Inc.
Supertec Machinery, Inc. Announces Earnings Results for the Full Year Ended December 31, 2018 
Supertec Machinery, Inc. announced earnings results for the full year ended December 31, 2018. For the full year, the company reported sales were TWD 386,339,000 compared with TWD 422,129,000 a year ago. Operating loss was TWD 23,609,000 compared with operating income of TWD 17,091,000 a year ago. Net loss was TWD 32,931,000 compared with net income of TWD 10,834,000 a year ago. Basic loss per share was TWD 1.19 compared with basic earnings per share of TWD 0.39 a year ago. 
**************************************************************************************
CID:  21129 Intelligent Systems Corporation
Intelligent Systems Corporation Announces Earnings Results for the Fourth Quarter and Full Year Ended December 31, 2018 
Intelligent Systems Corporation announced ear

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:356: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:

value_checks = source_data_copy[['CID', 'period', 'multiplierCode', 'value', 'con_value_fin', 'ComparisonPeriod', 'con_ComparisonPeriod_fin']]
value_checks.to_csv('value_checks.csv', index=False)